In [2]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.4 MB/s eta 0:00:00


In [14]:
import anthropic
from anthropic.types import MessageParam
from google.colab import userdata

## Tools

In [4]:
weather_tool = {
    "name": "get_current_weather",
    "description": "Get the current weather for the given location. The location should be a city with state/country. "
    + "Use this tool if the user needs to know the current weather for a location. "
    + "The tool will return the current temperature in the given unit. If the unit is not provided, it will default to celsius.",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state/country to get the weather for. e.g. 'New York, NY', 'San Francisco, CA', 'London, UK'",
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "The unit of temperature to use.",
            },
        },
        "required": ["location"],
    },
}

forecast_tool = {
    "name": "get_weather_forecast",
    "description": "Get up to 7-day weather forecast for the given location. The location should be a city with state/country. "
    + "Use this tool if the user needs to know the future weather/upcoming conditions for a location, e.g. if it will be rainy next week. "
    + "If the number of days is not specified, the tool will default to 7 days."
    + "The tool will return a string summarizing the forecast for the number of days specified.",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state/country to get the forecast for. e.g. 'New York, NY', 'San Francisco, CA', 'London, UK'",
            },
            "days": {
                "type": "integer",
                "description": "The number of days to get the forecast for (must be between 1 and 7)",
                "minimum": 1,
                "maximum": 7,
            },
        },
        "required": ["location"],
    },
}


def get_weather(location: str, unit: str = "celsiusn") -> str:
    def celsius_to_fahrenheit(celsius: float) -> float:
        return (celsius * 9 / 5) + 32

    if "San Francisco" in location:
        degrees = 15
    elif "New York" in location:
        degrees = 10
    elif "London" in location:
        degrees = 5
    else:
        return f"Weather information not available for {location}."

    if unit == "fahrenheit":
        return f"{celsius_to_fahrenheit(degrees)} degrees fahrenheit"
    else:
        return f"{degrees} degrees celsius"


def get_weather_forecast(location: str, days: int = 7) -> str:
    forcasts = {
        "San Francisco": "Mostly sunny with a chance of rain. Temperature will be around 60 degrees fahrenheit.",
        "New York": "Partly cloudy with a chance of snow. Temperature will be around 30 degrees fahrenheit.",
        "London": "Cloudy with a chance of rain. Temperature will be around 40 degrees fahrenheit.",
        "Seattle": "Rainy with a chance of snow. Temperature will be around 50 degrees fahrenheit.",
    }
    for key in forcasts.keys():
        if key in location:
            return forcasts[key]
    return f"Forecast not available for {location}."

## Tool call dispatcher

In [5]:
def process_tool_call(tool_name, tool_input) -> str:
    if tool_name == weather_tool["name"]:
        return get_weather(
            location=tool_input["location"],
            unit=tool_input["unit"] if "unit" in tool_input else "celsius",
        )
    elif tool_name == forecast_tool["name"]:
        return get_weather_forecast(
            location=tool_input["location"],
            days=tool_input["days"] if "days" in tool_input else 7,
        )
    else:
        return f"Error: Unknown tool name: {tool_name}."

## Agent loop

In [6]:
# ANSI color codes
CYAN = "\033[96m"
YELLOW = "\033[93m"
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"

In [ ]:
def run_agent(
    user_query: str,
    client: anthropic.Anthropic,
    model: str = "claude-3-haiku-20240307",
    system_message: str | anthropic.Omit = anthropic.Omit(),
    tools: list[dict] = [],
    max_interations: int = 10,
) -> tuple[str | None, list[MessageParam]]:
    current_iteration = 0
    messages = [{"role": "user", "content": user_query}]

    while current_iteration < max_interations:
        current_iteration += 1

        print(f"{CYAN}***LOG: sending messages: {messages}{RESET}")
        response = client.messages.create(
            model=model,
            max_tokens=1024,
            tools=tools,
            messages=messages,
            system=system_message,
        )
        print(f"{CYAN}***LOG: claude response: {response}{RESET}")
        print("--------------------------------")

        messages.append({"role": "assistant", "content": response.content})

        if response.stop_reason == "tool_use":
            tool_calls = [
                content for content in response.content if content.type == "tool_use"
            ]
            tool_results = []
            for tool_call in tool_calls:
                tool_name = tool_call.name
                tool_input = tool_call.input
                tool_output = process_tool_call(tool_name, tool_input)
                print(
                    f"{YELLOW}***LOG: calling tool {tool_name} with input {tool_input}; tool output: {tool_output}{RESET}"
                )
                tool_results.append(
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_call.id,
                        "content": tool_output,
                    }
                )
            tool_resp_message = {"role": "user", "content": tool_results}
            messages.append(tool_resp_message)

        elif response.stop_reason == "end_turn":
            break

        else:
            print(f"{RED}WARNING: Unexpected stop reason {response.stop_reason}{RESET}")
            break

    if current_iteration >= max_interations:
        print(f"{RED}WARNING: Max iterations reached. {RESET}")

    final_content = next(
        (content.text for content in response.content if content.type == "text"),
        None,
    )

    return final_content, messages

In [12]:
def test_weather(query: str):
    api_key = userdata.get("ANTHROPIC_API_KEY")
    client = anthropic.Anthropic(api_key=api_key)
    model = "claude-3-haiku-20240307"

    tools = [weather_tool, forecast_tool]
    tool_names = ", ".join(t["name"] for t in tools)
    system_message = f"""
You have access to the following tools: {tool_names}.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.
Include all tool calls in a single response whenever possible.
Prioritize calling tools in parallel over sequential execution whenever possible.

List of available tools: {tool_names}
    """
    print(f"system message: {system_message}")

    final_content, messages = run_agent(
        user_query=query,
        client=client,
        model=model,
        system_message=system_message,
        tools=tools,
        max_interations=10,
    )

    print()
    if final_content:
        print(f"{GREEN}Claude: {final_content}{RESET}")
    else:
        print(f"{RED}WARNING: No text response from Claude. {RESET}")

    print(f"\n{CYAN}--- CONVERSATION HISTORY ---{RESET}")
    for i, message in enumerate(messages):
        print(f"{CYAN}[{i}] {message['role']}: {message['content']}{RESET}")
    print(f"{CYAN}--- END OF CONVERSATION HISTORY ---{RESET}")

## Tests

In [16]:
test_weather("What's weather in NYC?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's weather in NYC?"}]
***LOG: claude response: Message(id='msg_011VpDaQtz9h4QfE4kfWNqTo', content=[TextBlock(citations=None, text="Okay, let's get the current weather for New York City:", type='text'), 

In [17]:
test_weather("What's weather in SFO and London?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's weather in SFO and London?"}]
***LOG: claude response: Message(id='msg_01QrAL2GfHYKe2uKpbHJaLjG', content=[TextBlock(citations=None, text="Okay, let's check the current weather for both San Francisco

In [18]:
test_weather("What's 2+2?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's 2+2?"}]
***LOG: claude response: Message(id='msg_01Q3Bs4hTMGhDSG4w2xu2mFM', content=[TextBlock(citations=None, text='4.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason

In [19]:
test_weather("What's capital of France?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's capital of France?"}]
***LOG: claude response: Message(id='msg_01A5SNQ1TGBrAMYByV1Av1W7', content=[TextBlock(citations=None, text='The capital of France is Paris.', type='text')], model='claude-3-hai

In [20]:
test_weather("Compare weather of 3 cities: SFO, NYC and London")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': 'Compare weather of 3 cities: SFO, NYC and London'}]
***LOG: claude response: Message(id='msg_01F3aLA72GBEXtUCZRbjBY9m', content=[TextBlock(citations=None, text="Okay, let's compare the weather for the 3 cit

In [21]:
test_weather("Is is warmer now in SF or NYC?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': 'Is is warmer now in SF or NYC?'}]
***LOG: claude response: Message(id='msg_016paoBKdo29pSm5mpHi1Z22', content=[TextBlock(citations=None, text='To compare the current weather in San Francisco (SF) and New Yo

In [25]:
test_weather("What's weather in Seattle?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's weather in Seattle?"}]
***LOG: claude response: Message(id='msg_01XPiwjBBhXYw5dbi2tLroRb', content=[TextBlock(citations=None, text='Okay, let me check the weather in Seattle for you:', type='text'), 

In [26]:
test_weather("Will it rain next week in Seattle?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': 'Will it rain next week in Seattle?'}]
***LOG: claude response: Message(id='msg_01BncoDa84gNj1wpkKHjLxEu', content=[TextBlock(citations=None, text="Okay, let's check the weather forecast for Seattle next wee

In [27]:
test_weather("What's the weather now in London, and will it rain next week?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': "What's the weather now in London, and will it rain next week?"}]
***LOG: claude response: Message(id='msg_01XYS4kvRSEduxiJ8U9w7zmL', content=[TextBlock(citations=None, text="Okay, let's check the current we

In [28]:
test_weather("Should I bring an umbrella to Seattle the day after tomorrow?")

system message: 
You have access to the following tools: get_current_weather, get_weather_forecast.
- ONLY use the tools listed above. DO NOT call any other tools that are not available to you.
- Choose the right tool based on the user's query.
- When no appropriate tool is available, respond directly with your own knowledge and expertise without invoking any tools.
- For maximum efficiency, whenever you perform multiple independent operations, invoke all relevant tools simultaneously rather than sequentially.  
Include all tool calls in a single response whenever possible. 
Prioritize calling tools in parallel over sequential execution whenever possible. 

List of available tools: get_current_weather, get_weather_forecast
    
***LOG: sending messages: [{'role': 'user', 'content': 'Should I bring an umbrella to Seattle the day after tomorrow?'}]
***LOG: claude response: Message(id='msg_011xjReSkidpKcUFkFgeNhYY', content=[TextBlock(citations=None, text="Okay, let's check the weather fo